# 🧠 QuanSkill – Personal Study Copilot (Assignment 01)

## ✅ Full Solution Notebook – **Open-Source Only**

This is the **reference solution** for **Assignment 01 – Personal Study Copilot (OSS)**.

- No OpenAI / paid keys required  
- Uses only **open-source models**  
- Follows the same structure as the assignment:
  1. Document ingestion & vector store  
  2. Retriever + `retrieve_context`  
  3. Open-source LLM + RAG chain  
  4. Simple conversational memory  
  5. Tool (calculator)  
  6. LangGraph router (`router → rag_node/tool_node → END`)  
  7. `chat_loop()` for interaction


## 0. Environment Setup

Uncomment and run the cell below **once** if these packages are not installed:

- `langchain`, `langchain-community`, `langgraph`  
- `faiss-cpu`  
- `transformers`, `accelerate`  
- `sentence-transformers`


In [1]:
# ⚙️ Install dependencies (only if needed)
!pip install -q langchain langchain-community langgraph
!pip install -q faiss-cpu
!pip install -q transformers accelerate sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 37.6 MB/s eta 0:00:00


## 1. Imports & Basic Configuration

In [2]:
from typing import List, Dict, Any
from dataclasses import dataclass, field
import os
import math
import re

# LangChain core
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# Vector store + embeddings
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline

# Text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangGraph
from langgraph.graph import StateGraph, END

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

## 2. Agent State (LangGraph)

In [3]:
@dataclass
class AgentState:
    user_input: str
    messages: List[Dict[str, Any]] = field(default_factory=list)
    route: str = ""          # 'rag' or 'tool'
    tool_result: Any = None  # output from tools

## 3. Study Documents (Self-Contained Demo)

We use `DOCS_FOLDER = "data/docs"`.  
If there are no `.txt` / `.md` files yet, we automatically create two small sample docs,
so the notebook always runs without FAISS errors.


In [4]:
DOCS_FOLDER = "data/docs"
os.makedirs(DOCS_FOLDER, exist_ok=True)

def ensure_sample_docs(folder: str):
    has_text_files = any(
        fname.lower().endswith((".txt", ".md"))
        for fname in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, fname))
    )
    if has_text_files:
        print("📚 Existing .txt/.md files found – using those as study docs.")
        return

    print("ℹ️ No .txt/.md found in", folder, "- creating sample docs for the demo.")
    sample_path_1 = os.path.join(folder, "agents_intro.txt")
    sample_path_2 = os.path.join(folder, "tools_and_rag.txt")

    with open(sample_path_1, "w", encoding="utf-8") as f:
        f.write(
            "This course explains how to build simple AI agents using LangChain and LangGraph. "
            "Students learn about prompts, vector stores, retrieval augmented generation (RAG), "
            "and basic routing between tools and retrievers."
        )

    with open(sample_path_2, "w", encoding="utf-8") as f:
        f.write(
            "Tools are functions that an agent can call to perform specific actions such as "
            "calculations or web searches. In a RAG system, the retriever tool is responsible "
            "for fetching relevant context from a vector store of embedded documents."
        )

ensure_sample_docs(DOCS_FOLDER)
print("✅ Study docs ready in", DOCS_FOLDER)

ℹ️ No .txt/.md found in data/docs - creating sample docs for the demo.
✅ Study docs ready in data/docs


### 3.1 Load Raw Documents

In [5]:
def load_raw_documents(docs_folder: str) -> List[str]:
    texts = []
    for fname in os.listdir(docs_folder):
        fpath = os.path.join(docs_folder, fname)
        if not os.path.isfile(fpath):
            continue
        if fname.lower().endswith(('.txt', '.md')):
            with open(fpath, 'r', encoding='utf-8', errors='ignore') as f:
                texts.append(f.read())
    return texts

raw_docs = load_raw_documents(DOCS_FOLDER)
print(f"Loaded {len(raw_docs)} raw documents.")

Loaded 2 raw documents.


### 3.2 Split into Chunks

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    add_start_index=True,
)

all_splits = text_splitter.create_documents(raw_docs)
print(f"Created {len(all_splits)} text chunks.")

if not all_splits:
    raise ValueError(
        "No text chunks created. Make sure there is at least one .txt or .md "
        f"file inside {DOCS_FOLDER} and rerun this cell."
    )

Created 2 text chunks.


## 4. Vector Store with Sentence-Transformers Embeddings

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore = FAISS.from_documents(all_splits, embeddings)
print("✅ Vector store created with", len(all_splits), "chunks.")

/tmp/ipython-input-2697131312.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store created with 2 chunks.


## 5. Retriever Helper – `retrieve_context`

In current LangChain versions, `as_retriever()` returns a **Runnable** retriever,  
so we use `.invoke(query)` instead of `.get_relevant_documents(...)`.


In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

def retrieve_context(query: str, k: int = 4) -> List[str]:
    """Return top-k relevant text chunks for a user query."""
    docs = retriever.invoke(query)
    return [d.page_content for d in docs]

# Quick sanity check
chunks = retrieve_context("What is this course about?")
if chunks:
    print("Sample retrieve:", chunks[0][:120], "...")
else:
    print("No chunks returned – check that your docs loaded correctly.")

Sample retrieve: This course explains how to build simple AI agents using LangChain and LangGraph. Students learn about prompts, vector s ...


## 6. Open-Source LLM + RAG Chain

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

gen_pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=False,          # no sampling, deterministic
    # ❌ no return_full_text here
)

llm = HuggingFacePipeline(pipeline=gen_pipe)
print("✅ FLAN-T5-small pipeline initialized.")

Device set to use cpu


✅ FLAN-T5-small pipeline initialized.


### 6.1 RAG Prompt + LCEL Chain

In [30]:
rag_prompt = ChatPromptTemplate.from_template(
    "You are a helpful study assistant.\n\n"
    "Use ONLY the following context to answer the question.\n"
    "If the answer is not in the context, say: 'I do not see this in the study materials.'\n\n"
    "Context:\n{context}\n\n"
    "Question:\n{question}\n\n"
    "Give a clear and concise answer."
)

def _lookup_context(inputs: Dict[str, Any]) -> Dict[str, Any]:
    query = inputs["question"]
    chunks = retrieve_context(query)
    return {
        "context": "\n\n".join(chunks),
        "question": query,
    }

lookup = RunnableLambda(_lookup_context)
rag_chain = lookup | rag_prompt | llm

def answer_with_rag(query: str) -> str:
    result = rag_chain.invoke({"question": query})
    return str(result).strip()

## 7. Simple Conversational Memory

In [31]:
conversation_history: List[Dict[str, str]] = []

def remember_message(role: str, content: str):
    conversation_history.append({"role": role, "content": content})

def get_formatted_history() -> str:
    lines = []
    for msg in conversation_history:
        lines.append(f"{msg['role'].upper()}: {msg['content']}")
    return "\n".join(lines)

## 8. Tool – Calculator

In [32]:
def calculator_tool(expression: str) -> str:
    try:
        allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"Result: {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

#print(calculator_tool("2 + 3 * 4"))

## 9. Router Node – Decide RAG vs Tool

In [33]:
def router_node(state: AgentState) -> AgentState:
    query = state.user_input.strip()
    if re.search(r"[0-9]", query) and re.search(r"[+\-*/]", query):
        state.route = "tool"
    else:
        state.route = "rag"
    return state

## 10. RAG Node – Study Questions

In [34]:
def rag_node(state: AgentState) -> AgentState:
    query = state.user_input
    remember_message("user", query)
    answer = answer_with_rag(query)
    remember_message("assistant", answer)
    state.messages.append({"role": "user", "content": query})
    state.messages.append({"role": "assistant", "content": answer})
    print(f"Agent (RAG): {answer}")
    return state

## 11. Tool Node – Utility / Math

In [36]:
def tool_node(state: AgentState) -> AgentState:
    query = state.user_input
    remember_message("user", query)
    tool_output = calculator_tool(query)
    state.tool_result = tool_output
    remember_message("assistant", tool_output)
    state.messages.append({"role": "user", "content": query})
    state.messages.append({"role": "assistant", "content": tool_output})
    print(f"Agent (TOOL): {tool_output}")
    return state

## 12. Build the LangGraph Agent

In [37]:
def router_edge(state: AgentState) -> str:
    if state.route == "rag":
        return "rag_node"
    if state.route == "tool":
        return "tool_node"
    return "rag_node"

graph_builder = StateGraph(AgentState)
graph_builder.add_node("router", router_node)
graph_builder.add_node("rag_node", rag_node)
graph_builder.add_node("tool_node", tool_node)

graph_builder.set_entry_point("router")
graph_builder.add_conditional_edges("router", router_edge)
graph_builder.add_edge("rag_node", END)
graph_builder.add_edge("tool_node", END)

app = graph_builder.compile()
print("✅ LangGraph compiled successfully.")

✅ LangGraph compiled successfully.


## 13. Chat Loop – Talk to the Copilot

In [38]:
print(answer_with_rag("What is this course about?")[:300])

How to build simple AI agents using LangChain and LangGraph


In [39]:
def chat_loop():
    print("🔹 Personal Study Copilot (OSS) – type 'exit' or 'quit' to stop.")
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break
        state = AgentState(user_input=user_input, messages=[])
        _ = app.invoke(state)

# Uncomment to start chatting after running all cells above:
chat_loop()

🔹 Personal Study Copilot (OSS) – type 'exit' or 'quit' to stop.

You: what is this course about?
Agent (RAG): How to build simple AI agents using LangChain and LangGraph

You: 2 + 3 * 4
Agent (TOOL): Result: 14

You: exit
👋 Goodbye!


## 14. How to Use This Solution with the Assignment Notebook (Instructions for PTV)

- Give students the **Assignment_01_OSS** notebook (with TODOs).  
- Keep this notebook as your **solution reference**.  
- When grading, check that they implemented the same high‑level pipeline:
  - docs → chunks → FAISS → retriever → RAG chain → tools → LangGraph router.
